In [8]:
#Load the required packages
import numpy as np
import pandas as pd
import multiprocessing as mp
import time

from subprocess import check_output
import subprocess
files=check_output(["ls", "raw_data"]).decode("utf8")
print(files)
#check the number of cores
num_cores = mp.cpu_count()
print("This kernel has :",num_cores,"cores")
#Check the number of row of each file
for file in files.split("\n"):
    path='raw_data/'+file
    popenobj=subprocess.Popen(['wc', '-l', path], stdout=subprocess.PIPE, 
                                              stderr=subprocess.PIPE)
    result,error= popenobj.communicate()
    #print(result,error)
    #print("The file :",file,"has :",result.strip().split()[0],"rows")


members_v3.csv
sample_submission_v2.csv
sample_submission_zero.csv
train.csv
train_v2.csv
transactions.csv
transactions_v2.csv
user_logs.csv
user_logs_v2.csv

This kernel has : 4 cores
The file : members_v3.csv has : b'6769474' rows
The file : sample_submission_v2.csv has : b'907472' rows
The file : sample_submission_zero.csv has : b'970961' rows
The file : train.csv has : b'992932' rows
The file : train_v2.csv has : b'970961' rows
The file : transactions.csv has : b'21547747' rows
The file : transactions_v2.csv has : b'1431010' rows
The file : user_logs.csv has : b'392106544' rows
The file : user_logs_v2.csv has : b'18396363' rows


IndexError: list index out of range

# user logs have 392 million records!!

The kaggle kernal limitations are as follows:
- It has a self timeout after processing a "cell" for ~10 minutes
- Memory limit is 8 GB 
Link:https://www.kaggle.com/wiki/Scripts

Hence by splitting the file into chunks less than 8 GB (I've used a chunk size of 40 mil rows), we can squeeze in the feature extraction with the timeout of 10 minutes.


In [2]:
#writing as a function
def process_user_log(chunk):
    grouped_object=chunk.groupby(chunk.index,sort=False) # not sorting results in a minor speedup
    func = {'date':['min','max','count'], 
           'num_25':['sum'],'num_50':['sum'],
           'num_75':['sum'],'num_985':['sum'],
           'num_100':['sum'],'num_unq':['sum'],'total_secs':['sum']}
    answer=grouped_object.agg(func)
    return answer
print("done")

done


In [14]:
reader = pd.read_csv('raw_data/user_logs_v2.csv', engine='c', index_col=['msno'])
result=process_user_log(reader)

In [15]:
print(len(result))
check=result.index.unique()
print(len(check))


1103894
1103894


In [16]:
#result.columns.values
result.columns = ['_'.join(col).strip() for col in result.columns.values]
result.columns

Index(['date_min', 'date_max', 'date_count', 'num_25_sum', 'num_50_sum',
       'num_75_sum', 'num_985_sum', 'num_100_sum', 'num_unq_sum',
       'total_secs_sum'],
      dtype='object')

In [17]:
result.head()
func = {'date_min':['min'],'date_max':['max'],'date_count':['count'] ,
           'num_25_sum':['sum'],'num_50_sum':['sum'],
           'num_75_sum':['sum'],'num_985_sum':['sum'],
           'num_100_sum':['sum'],'num_unq_sum':['sum'],'total_secs_sum':['sum']}
processed_user_log=result.groupby(result.index).agg(func)
print(len(processed_user_log))
processed_user_log.columns = processed_user_log.columns.get_level_values(0)
print(processed_user_log.head())

1103894
                                              date_min  date_max  date_count  \
msno                                                                           
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=  20170301  20170331           1   
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=  20170301  20170331           1   
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=  20170301  20170331           1   
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=  20170302  20170330           1   
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=  20170301  20170331           1   

                                              num_25_sum  num_50_sum  \
msno                                                                   
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=          86          11   
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=         191          90   
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=          43          12   
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=         207         163   

In [18]:
processed_user_log.head()

,date_min,date_max,date_count,num_25_sum,num_50_sum,num_75_sum,num_985_sum,num_100_sum,num_unq_sum,total_secs_sum
msno,,,,,,,,,,
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20170301,20170331,1,86,11,10,5,472,530,117907.425
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,20170301,20170331,1,191,90,75,144,589,885,192527.892
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20170301,20170331,1,43,12,15,12,485,468,115411.260
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,20170302,20170330,1,207,163,100,64,436,828,149896.558
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,20170301,20170331,1,105,24,39,35,479,230,116433.247


In [ ]:
# this function did not work because apparently only lists can be passed back from a function through the 
# multiprocessing package 
# If we use multiple cores the process can be 16 times faster

# create as many processes as there are CPUs on your machine
# if __name__ == '__main__':
#     size=1000000
#     parser = pd.read_csv('../input/user_logs.csv',chunksize=size,index_col=['msno'])
#     num_processes = mp.cpu_count()
#     pool=mp.Pool(processes=num_processes)
#     start_time = time.time()
#     list_of_ans=[]
#     def collect_results(result):
#         list_of_ans.extend(result)
#         print(done)
#     for i,df in enumerate(parser):
#         print("starting at",i," %s seconds" % (time.time() - start_time))
#         pool.map(process_user_log,df,callback=collect_results) 
#         
#     print("action ",i,"took %s seconds" % (time.time() - start_time))
#     pool.close()
#     pool.join()
        


In [20]:
processed_user_log.to_csv("cleaned_input/processed_user_log_v2.csv")